<center><H1>Creación de modelos neuronales en</H1><center>

<center><img src="https://www.gstatic.com/devrel-devsite/prod/ve2848ad92313fddfcd40baeb58a2f663fe2fd55c371a714a6bb3e329e2b15223/tensorflow/images/lockup.svg"  height="80px" style="padding-bottom:5px;"  /></center>

<center><H2>Julio Waissman Vilanova</H2>

<table align="center">
      <td align="center"><a target="_blank" href="https://www.unison.mx">
            <img src="https://www.unison.mx/wp-content/themes/awaken/images/logo.png"  height="70px" style="padding-bottom:5px;"  /></a></td>  
      <td align="center"><a target="_blank" href="https://www.gob.mx/cenace">
            <img src="https://universidad.cenace.gob.mx/pluginfile.php/244/block_html/content/CENACE-logo-completo.png" width="300" style="padding-bottom:5px;" /></a></td>
      <td align="center"><a target="_blank" href="https://colab.research.google.com/github/juliowaissman/rn-cenace/blob/main/modelos_tensorflow.ipynb">
            <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecuta en Google Colab</a></td>

</table>

In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pprint

# 1 Definicion de redes simples con `Sequential`

## 1.1 ¿Cuando usar un modelo tipo `Sequential`?


Un modelo basado en `Sequential` es apropiado cuando:

- Se tiene un modelo de capas en forma de una pila sencilla
- Cada capa tiene exactamente un tensor de entrada y un tensor de salida

Por ejemplo:

In [ ]:
# Modelo con 3 capas
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="capa1"),
        layers.Dense(3, activation="relu", name="capa2"),
        layers.Dense(4, name="capa3"),
    ]
)


# Prueba el modelo
x = tf.ones((3, 3))
y = model(x)
print(f"La salida es:\n {y.numpy()}")
print("\nY el modelo es el siguiente:")
model.summary()

In [ ]:
# Otra forma de usar Sequential
layer1 = layers.Dense(2, activation="relu", name="capa1")
layer2 = layers.Dense(3, activation="relu", name="capa2")
layer3 = layers.Dense(4, name="capa3")

model = keras.Sequential(name="Mi Modelo")
model.add(layer1)
model.add(layer2)
model.add(layer3)


# Prueba el modelo
x = tf.ones((3, 3))
y = model(x)
print(f"La salida es:\n {y.numpy()}")
print("\nY el modelo es el siguiente:")
model.summary()

Se pueden ir sacando capas de la pila utilizando el método `pop`

In [ ]:
model.pop()
model.summary()

`Sequential` es la forma más sencilla de definir un modelo neuronal, por lo que no es apropiado para modelos que:

- Tienen multiples entradas y salidas
- Alguna capa tiene multiples entradas o multiples salidas
- Se necesita compartir alguna capa
- Una topología no lineal (i.e. resnet)



## 1.2 Especificar el `shape` de la entrada al principio

Casi todos los modelos de capas y modelos desarrollados en keras se inicializan hasta que conocen su entrada (o la dimensión de esta). Por ejemplo, la siguiente capa tiene su vector de pesos vacío hasta que no se usa por primera vez:

In [ ]:
layer = layers.Dense(3)
layer.weights  

In [ ]:
# Call layer on a test input
x = tf.ones((1, 4))
y = layer(x)
layer.weights

Esto pasa típicamente con los modelos basados en `Sequential`donde, mientrs no se defina la entrada, no se inicializan los pesos, y varias de las funciones no pueden usarse.

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
) 

# En este punto, no hay pesos
#print(f"Los pesos existentes son: {model.weights}") 

# Tampoco puedes hacer este
#display(model.summary())

# Pero si se inicializa la coosa cambia
x = tf.ones((1, 4))
y = model(x)
print(model.weights) 
display(model.summary())


Con el fin que el modelo se encuentre bien definido desde el principio, así como la asignación inicial de pesos y sesgos, se recomienda inicializar los modelos con el tamaño de la entrada. Esto se puede hacer de dos formas:

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()

# 2 Entrenamiento y evaluación básicos de redes neuronales

# 2.1 Introducción

Aqui vamos a ver como utilizar las herramientas de entrenamiento, evaluación
y predicción que vienen incluidas en las APIs de las formas de desarrollar modelo
vistas en la libreta anterior y que pueden ser llamadas como `Model.fit()`,
`Model.evaluate()` and `Model.predict()`. 

Si buscamos modificar el proceso de entrenamiento y evaluación a la medida, 
se pueden consultar las siguientes guías:

1. Para desarrollar un paso de aprendizaje a la medida, se puede consultar la guía
de Tensorflow 2.x [Customizing what happens in `fit()` guide](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit/).

2. Para desarrollar un ciclo de entrenamiento y evaluación diferente, consultar la guía
["writing a training loop from scratch"](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch/).

3. Para entrenamiento distribuido, ver la guía [guide to multi-GPU & distributed training](https://keras.io/guides/distributed_training/).

Las guías en español parecen traducidas directamente con el google translate, 
por lo que recomiendo leerlas en su versión en inglés. 

Para ilustar el uso del aprendizaje y validación, vamos a utilizar el conjunto de datos 
más visto en el mundo del aprendizaje profundo: la base de datos de [MNIST](http://yann.lecun.com/exdb/mnist/). Afortunadamente es tan común que ya viene por default en Tensorflow y es fácil de obtener

In [ ]:
(x_train_original, y_train), (x_test_original, y_test) = keras.datasets.mnist.load_data()

# Preprocesamiento (aplanado de los datos) 
x_train = x_train_original.reshape(60000, 784).astype("float32") / 255
x_test = x_test_original.reshape(10000, 784).astype("float32") / 255

# Las salidas como flotantes para ser usadas por Tensorflow
y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserva 10,000 muestras para validación
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

Vamos a ver algunos de los datos de entrada

In [ ]:
indices = np.random.randint(0, 50000, size=16)

pl.figure(figsize=(12,12))
for i in range(16):
    pl.subplot(4, 4, i+1)
    pl.imshow(x_train_original[indices[i],:,:])
    pl.axis('off')
    pl.title(f"Salida = {int(y_train[indices[i]])}")
pl.show()

Y ahora, pues vamos a definir un modelo neuronal sencillo para practicar

In [ ]:
model = keras.Sequential(name="MiModeloMNIST")
model.add(keras.Input(shape=(784,), name="digitos"))
model.add(layers.Dense(64, activation="relu", name="capa_oculta_1"))
model.add(layers.Dense(64, activation="relu", name="capa_oculta_2"))
model.add(layers.Dense(10, activation="softmax", name="prediccion"))

model.summary()

## 2.2 El método `compile` 

El método `compile`se utiliza para especificar pérdida, métrica y método de optimización

Vamos a aplicarlo primero y lo vamos desglosando paso a paso:

In [ ]:
model.compile(
    # Método de optimización a utilizar
    optimizer=keras.optimizers.RMSprop(),  
    # Funcion de pérdida a minimizar
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # Lista con métricas para monitorear
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

Si queremos utilizar diferentes métodos es mejor separar el modelo no compilado del modelo compilado usando funciones:

In [ ]:
def modelo_no_compilado():
    model = keras.Sequential(name="MiModeloMNIST")
    model.add(keras.Input(shape=(784,), name="digitos"))
    model.add(layers.Dense(64, activation="relu", name="capa_oculta_1"))
    model.add(layers.Dense(64, activation="relu", name="capa_oculta_2"))
    model.add(layers.Dense(10, activation="softmax", name="prediccion"))
    return model

def modelo_compilado(
                optimizador=keras.optimizers.RMSprop(), 
                perdida=keras.losses.SparseCategoricalCrossentropy(), 
                metricas=[keras.metrics.SparseCategoricalAccuracy()]):
    model = modelo_no_compilado()
    model.compile(
        optimizer=optimizador,
        loss=perdida,
        metrics=metricas,
    )
    return model


In [ ]:
modelo = modelo_compilado()

Para especificar los posibles optimizadores, funciones de pérdida y métricas que vienen ya desarrolladas por *Keras* se puede consultar la [API de Keras para métodos de optimización](https://keras.io/api/optimizers/), la [API de Keras para funciones de pérdida](https://keras.io/api/losses/) y la [API de Keras para métricas](https://keras.io/api/metrics/).

## 2.3 Aplicando `fit`, `evaluate` y `predict`

Ya con un modelo compilado podemos pasar a la fase de entrenamiento y evaluación del modelo, para lo que vamos a usar las funciones `fit`y `evaluate`. Empecemos con `fit`:

In [ ]:
num_epochs = 4
print(f"Entrenamiento del modelo en {num_epochs} epochs")
history = modelo.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=num_epochs,
    # Datos de validación para medir el aprendizaje al final de cada epoch
    validation_data=(x_val, y_val),
)

En `history` se tiene información de la fase de aprendizaje

In [ ]:
pprint.pprint(history.history)

Si los datos de entrenamiento se mantienen en `ndarray`de numpy (como en este caso),
entonces es posible no necesiotar separar explicitamente los datos de entrenamiento
y validación, si no simplemente decir en `fit` que porcentaje de los datos deben ser
utilizados para validación y el método se encarga de dividirlos.

Por ejemplo, si se quisiera utilizar el 20% de los datos para validación se podría
utilizar el siguiente código (asumiendo que no los hubieramos separado ya los datos)

```python
modelo = modelo_compilado()
modelo.fit(x_train, y_train, batch_size=64, validation_split=0.2, epochs=1)
```

Para evaluar el modelo, vamos a revisar las métricas establecidas pero con los datos de prueba usando `evaluate`:

In [ ]:
print("Evaluando en los datos de prueba")

results = model.evaluate(x_test, y_test, batch_size=128)

print("\n\nPérdida en test, Accuracy en test:", results)

Por último, vamos a predecir los valores en los primeros 3 datos del conjunto de test:

In [ ]:
print("Prediciendo para 3 ejemplos")
predict = model.predict(x_test[:3])
print("Shape de predict:", predict.shape)
print("\nY las predicciones son:")
print(predict)
print("\nY si queremos las clases, pues usamos `argmax` de numpy")
print(predict.argmax(axis=1))

## 2.4 Guardando y cargando modelos

¿Para que entrenar un modelo si no lo vamos a volver a usar? Para guardar un modelo entrenado (y volverlo a cargar) Keras ofrece una interface, en la cual se guarda en un único archivo toda la información sobre un modelo y se puede volver a cargar en forma transparente.

Guardemos y carguemos de nuevo nuestro modelo:

In [ ]:
modelo.save("modelito.h5")

In [ ]:
otro_modelo = keras.models.load_model("modelito.h5")

print("Prediciendo para los mismos 3 ejemplos")
predict = otro_modelo.predict(x_test[:3])
print("Shape de predict:", predict.shape)
print("\nY las predicciones son:")
print(predict)
print("\nY si queremos las clases, pues usamos `argmax` de numpy")
print(predict.argmax(axis=1))

Cuando entrenamos con modelos muy complicados o muchos datos, es posible que algo pase y se nos pierda lel entrenamiento a la mitad, o peor aun, casi terminandolo, lo que es una desgracia. Afortunadamente en Keras es posible ir generando archivos de checkpoint como sigue:

```python

modelo = modelo_compilado()

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="mymodel_{epoch}",
        save_best_only=True,  # Only save a model if `val_loss` has improved.
        monitor="val_loss",
        verbose=1,
    )
]
model.fit(
    x_train, y_train, epochs=2, batch_size=64, callbacks=callbacks, validation_split=0.2
)
```

Intenta probarlo y revisa que resultados te da y donde se guardan los archivos de checkpoint